In [16]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import dependencies
# data cleaning and manipulation 
import pandas as pd
import numpy as np

# data visualization
import seaborn as sns

# machine learning
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as skl_lm
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [17]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [18]:
df1 = pd.read_csv('/kaggle/input/toronto-airbnb-dataset/listings_sep_09_2020.csv', delimiter=',')
df1.dataframeName = 'listings_sep_09_2020.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

#Merging list of districts acquired from Tableau data exploration that Roma and Mariana webscraped
new_city_df = pd.read_csv('/kaggle/input/torontodistricts/city_designated_neighbourhoods_c.csv', delimiter=',')
df2 = pd.merge(df1, new_city_df, on="neighbourhood_cleansed")
print(f'There are {nRow} rows and {nCol} columns')



In [19]:
df1['neighbourhood_cleansed'].head(4)

In [20]:
df2[['price','review_scores_rating','neighbourhood_cleansed','former_city']].sample(10)

In [21]:
#74 features
(round(df2.isnull().sum() / df2.shape[0] *100,2)).sort_values(ascending=False)

In [22]:
df2.info()

 # **Overview of the Toronto Airbnb data and the data types along with missing values**

In [23]:
#Information on numeric and string varaibles
#int variable type is for numeric, float is numeric, object is for string
#There are 19343 entries, however some features have missing entries
#Example,nieghbourhood has 713 rows instead of 1000 rows, meaning rest are NA values
#Target variables, price has 1000 entries; not missing any entries
#Potential target variable, review scores rating has 874 ratings, so review scores rating is missing 126 entries.

df2.info()

In [24]:
#Looking at variables might not need as are redundant, have all NaN values or not relevant to business objective implementation of machine learning regression model
df2[['host_listings_count','neighbourhood_cleansed','neighbourhood_group_cleansed','host_neighbourhood','neighbourhood','host_location','host_identity_verified','license', 'calendar_updated','review_scores_value', 'review_scores_rating']]

In [25]:
df2.drop(['cdn_num','Unnamed: 3','Unnamed: 4'],axis = 1, inplace=True)

In [26]:
#74 features
df2.info()

# Data Wrangling: Removing Features that do not need as are redundant, have all NaN values or not relevant to business objective

In [27]:
#Removing variables that do not need as are redundant, have all NaN values or not relevant to business objective
#Some categorical variables are not necessary and also some text for categorical variables which are actually lists such as amenities, host_verifications will require some extensive preformatting
#drop columns "host_since", "host_location", "host_verifications", "host_identity_verified", "has availability", "first_review", "last_review", "ameneties"
#Also some columns such as maximim_minimum_nights are redundant
df3 = df2.drop(['listing_url','scrape_id','last_scraped','name','description','host_listings_count','neighborhood_overview','picture_url','host_id','host_url','host_name','host_about','host_thumbnail_url','host_picture_url','host_neighbourhood','host_has_profile_pic','neighbourhood','neighbourhood_group_cleansed','bathrooms','calendar_updated','license','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_last_scraped','host_location','host_verifications','host_identity_verified','amenities','has_availability','first_review','last_review','property_type', 'bathrooms_text'], axis = 1)


In [28]:
#Data types of each column
#Object refers to string, int and float refer to numeric

#Target variables in our two prediction models are "Price" and "review_scores_ratings"
#Price has to be converted to numeric, take out the dollar sign
#Should only have variables relevant to business objective and variables that have less than 40% NaN values.

#Left with 35 features out of 73 features

df3.info() 
# Order the values in terms of percentage of dtype
(round(df3.isnull().sum() / df3.shape[0] *100,2)).sort_values(ascending=False)

In [29]:
df3.sample(2)

In [30]:
#dataframe layout
df3.shape

In [31]:
print("Columns with more than 40% missing values: \n",df3.columns[df3.isnull().mean() > 0.40].values)

In [32]:
pr = len(pd.unique(df3['price']))
re = len(pd.unique(df3['review_scores_rating']))
print(pr, "unique prices", re, "unique ratings")

In [33]:
df3['price'].unique()

In [34]:
# Reformat price column values to remove , and $ if any
df3['price'] =  df3["price"].astype('str').map(lambda x: x.replace("$",'').replace(",",''), na_action = 'ignore').astype(float)

In [35]:
df3['price']

# Data Wrangling and Data Cleaning: 
# 1. Identifying features with NaN values (completed)
# 2. Feature Engineering (Completed)
# 3. Imputing the NaN values with prediction method (Completed)
# 4. Outliers to remove? Data with clear outliers is winsorized or log transformed (Completed)
# 5. Dummy coding (completed)



In [36]:
#Columns with missing values
print("Column names with missing values: \n", df3.columns[df3.isnull().sum(axis=0)>0].values)

#Total no of missing values in df_listings dataset
print("Total number of missing values are:",df3.isnull().sum().sum())

In [37]:
print("Columns with more than 40% missing values: \n",df3.columns[df3.isnull().mean() > 0.40].values)

In [38]:
#Left with 36 features to work with, price and review scores rating being target variables
#14 variables of the type float, 14 variables of type int and 8 variables of object
df3.info()

In [39]:
df3.isnull().sum()

In [40]:
#host is superhost and acceptance rate are categorical variables that has to be imputed and converted
df = df3[['room_type','host_since','host_response_time','host_is_superhost','host_acceptance_rate', 'host_response_rate']]

In [41]:
df.head(3)

In [42]:
#Applying imputer to replace NaN values in categorical variables: host response time, host response rate, host acceptance rate with most frequent
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df = pd.DataFrame(imp.fit_transform(df),columns=df.columns, index=df.index)


In [43]:
df.head(3)

In [44]:
df3=df3.drop(['room_type','host_since','host_response_time','host_is_superhost','host_response_rate','host_acceptance_rate'],axis=1)

In [45]:
df3.head(4)

In [46]:
frames = [df3, df]  
df4=pd.concat(frames, axis =1)

In [47]:
df4.head(10)

In [48]:
df4.info()

In [49]:
df4[['host_since','host_acceptance_rate','host_response_rate']]

In [50]:
#Take out perentage from host acceptance rate and convert to integer
#Remove % sign and convert the data type to int
#df_listings['host_response_rate'] = (df_listings['host_response_rate'].str[:-1].astype(int))
#df_listings['host_acceptance_rate'] = (df_listings['host_acceptance_rate'].str[:-1].astype(int))
df4['host_response_rate'] = (df4['host_response_rate'].str[:-1].astype(int))
df4['host_acceptance_rate'] = (df4['host_acceptance_rate'].str[:-1].astype(int))

In [51]:
df4.head(4)

# 2. Imputing the NaN values with prediction method (Completed)

In [52]:
df4_numerical = df4[['bedrooms','beds','host_total_listings_count','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','reviews_per_month']]
 

In [53]:
df4_nonmissing = df4.drop(['bedrooms','beds','host_total_listings_count','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','reviews_per_month'], axis = 1)


In [54]:

#There are some numerical columns that need to imputed

#Inmputing missing values in numerical columns using the MICE method

#Review ratings score had missing values

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from fancyimpute import IterativeImputer

#calling the MICE class
mice_imputer = IterativeImputer(random_state=42)

#imputing the missing value with mice imputer
imputed = mice_imputer.fit_transform(df4_numerical)
df4_numerical_imputed = pd.DataFrame(imputed, columns= df4_numerical.columns)
round(df4_numerical_imputed, 2)

In [55]:
df4_nonmissing.info()

In [56]:
df4_numerical_imputed.info()

In [57]:
#df4_numerical_imputed.to_csv('Toronto_Airbnb_imputednumerical_dataset.csv',index=False)
#df4_nonmissing.to_csv('Toronto_Airbnb_nonmissing_dataset.csv',index=False)

frames = [df4_numerical_imputed, df4_nonmissing]
df4_cleaned = pd.concat(frames, axis=1)



In [58]:
df4_cleaned.info()

In [59]:
df4_cleaned['price'].describe()

In [60]:
df5=df4_cleaned

In [61]:
df5.info()

# 3. Outliers removed using IQR method (Completed)

In [62]:
df5.describe()

In [63]:
df5.hist(bins=15, figsize=(20,20));

In [64]:
# Reformat price column values to remove , and $ if any
df2['price'] =  df2["price"].astype('str').map(lambda x: x.replace("$",'').replace(",",''), na_action = 'ignore').astype(float)

In [65]:
df5_cat = df5.select_dtypes('object')
df5_num = df5.select_dtypes(np.number)

#Seems for most of the data, there are outliers.
#Outliers present in data
#Even for price, there are outliers
#From Univariate analysis what can be inferred in terms of distribution of data and outliers?
# Numerical columns
for i in df5_num:
    fig,ax= plt.subplots(1,2)
    print(i)
    sns.distplot(df5_num[i],ax=ax[0],color='Green')
    sns.boxplot(x = df5_num[i],ax=ax[1],palette='Greens')
    #sns.violinplot(df_num[i],ax=ax[2],palette='Greens')
    plt.show()

In [66]:
df5[['price','bedrooms', 'host_acceptance_rate','host_response_rate', 'review_scores_rating','reviews_per_month','beds', 'accommodates', 'minimum_nights','maximum_nights','calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']].describe()


In [67]:
#As can be seen most of the data contained within features was positively or negatively
#Several outliers are present within each feature
#The outliers were noted from use of describe function and confirmed from the box plots and visualization of the distributions
#For example, for price, the maximium value is $13164, but the average price was $141.28 and 75% are below $150
#Similarly for minimum_nights, the maximum value was 1125, but the average was 10 and 75% below 5
#For both bedrooms and beds, mean was 1,39 and 1.63 respectively; maximum being 16 and 17.
#For scores for review rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_value, reviews_per_month
#there were clear outliers in the data
#Clear outliers were present in price, accomodates, host_total_listings_count, reviews per month, bedrooms, beds, minimum_nights, maximum_nights, number of reviews, number_of_review_ltm


#Three options for removing outliers were available to us, the IQR method, Winsorize and Log transform
#Log transformation reduces the skewness of data and tries to make it normal. However we had zero or negative values contained within some of our features.
#With winsorizing, any value of a variable above or below a percentile k on each side of the variables’ distribution is replaced with the value of the k-th percentile itself. 
#For example, 90% winsorization means the replacement of the top 5% and bottom 5% of the data. The top 5% of the data is replaced by the value of the data at the 95th percentile and the value of the bottom 5% of the data is replaced by the value of the data at the 5th percentile. 
#However, winsorization would not be feasible as would not be applied properly to particular features in our data and would replace values
#For the above reasons, we selected the Interquartile range (IQR) method. For the IQR method, the third quartile (75th percentile) and first quartile (25th percentile) is subtracted to get the IQR. Any numbers less than the First quartile subtracted from 1.5xIQR is considered an outlier and removed; whereas any number
#greater than the third quartile substracted from 1.5xIQR is considered an outlier and removed.


In [68]:
df5.info()

In [69]:
#Need id from df5
#As removing outliers, but don't want to include ID, longitude and latitude
dfid =df5[['id','longitude','latitude']]


In [70]:
df5.drop(['id','longitude','latitude'], axis = 1, inplace = True)

In [71]:
df5.info()

In [72]:
 #https://stackoverflow.com/questions/34782063/how-to-use-pandas-filter-with-iqr
def mod_outlier(df):
        df1 = df.copy()
        df = df._get_numeric_data()

        q1 = df.quantile(0.25)
        q3 = df.quantile(0.75)

        iqr = q3 - q1

        lower_bound = q1 -(1.5 * iqr) 
        upper_bound = q3 +(1.5 * iqr)


        for col in df.columns:
            for i in range(0,len(df[col])):
                if df[col][i] < lower_bound[col]:            
                    df[col][i] = lower_bound[col]

                if df[col][i] > upper_bound[col]:            
                    df[col][i] = upper_bound[col]    


        for col in df.columns:
            df1[col] = df[col]

        return(df1)

In [73]:
df5.info()

In [74]:
df = mod_outlier(df5)

In [75]:
df4[['price', 'review_scores_rating', 'minimum_nights', 'host_total_listings_count']].describe()

In [76]:
df[['price', 'review_scores_rating', 'minimum_nights', 'host_total_listings_count']].describe()

# 4. Feature Engineering (Completed)

In [77]:
#Binning
df['labels_host_acceptance_rate'] = pd.cut(x=df4['host_acceptance_rate'], bins=[-1, 25, 50, 75, 100], labels=['Very strict', 'Strict', 'Accepting', 'Very Accepting'])
df[['host_acceptance_rate','labels_host_acceptance_rate']].head(10)


In [78]:
df['host_response_rate']

In [79]:
df['labels_host_response_rate'] = pd.cut(x=df['host_response_rate'], bins=[-1, 25, 50, 75, 100], labels=['Very low', 'low', 'high', 'Very high'])
df[['host_response_rate','labels_host_response_rate']].head(10)

In [80]:
df['host_response_time'].head(5)

In [81]:
df['host_since_year'] = pd.DatetimeIndex(df['host_since']).year


In [82]:
df['host_since_year'].head(5)

In [83]:
df[['host_acceptance_rate','labels_host_acceptance_rate','host_response_rate','labels_host_response_rate','host_since_year']]

In [84]:
df.info()

In [85]:
df['host_length']=2021-df['host_since_year']

In [86]:
df.info()

In [87]:
frames = [dfid, df]  
df_final_tableau=pd.concat(frames, axis =1)

In [88]:
df_final_tableau.head(2)

In [89]:
df_final_tableau.info()

In [90]:
df_final_tableau.to_csv('Toronto_Airbnb_final_cleaned_dataset.csv',index=False)

In [91]:
#Final dataframe
#df_cleaned = df[['host_since_year','host_length','host_acceptance_rate','host_response_time','host_is_superhost','host_total_listings_count','latitude','longitude','room_type','accommodates','bedrooms','beds','price','minimum_nights','maximum_nights','availability_30','availability_60','availability_90','availability_365','number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','reviews_per_month','neighbourhood_cleansed','former_city','labels_host_acceptance_rate','labels_host_response_rate']]
df_cleaned_machinelearning = df_final_tableau.drop(['id','neighbourhood_cleansed'], axis = 1)

In [92]:
#There are 39 features in the final data frame
df_cleaned_machinelearning.info()

In [93]:
df_cleaned_machinelearning.to_csv('Toronto_Airbnb_final_cleaned_machinelearning_dataset.csv',index=False)

# 5. Dummy code variables for Machine Learning

In [94]:
df_cleaned_machinelearning.info()

In [95]:
df_cleaned_machinelearning.head(5)

In [96]:
df=df_cleaned_machinelearning.drop(['longitude','latitude','labels_host_response_rate'], axis = 1)

In [97]:
df.info()

# Explanatory Analysis on the Clean Data

In [98]:
df.head(2)

In [99]:
# calculate average price by listing in each neighbourhood
df.groupby('former_city').mean().sort_values('price', ascending = False)[['price']]
#Highest:Bridle Path-Sunnybrook-York Mills	850.000000
#Lowest:New Toronto	35.500000
#Why different from what is shown in Tableau?

In [100]:
#Calculate average review rating by listing in each neighborhood
df.groupby('former_city').mean().sort_values('review_scores_rating', ascending = False)[['review_scores_rating']]

In [101]:
#Find the room type distribution percentage
df_room = pd.DataFrame()
df_room['Room_Type_count'] = df.room_type.value_counts(ascending = False)
df_room['Room_Type_percentage'] = round(df.room_type.value_counts(ascending = False)/df2.shape[0]*100, 2)
df_room

In [102]:
#Look at the correlation between numerical variables to see what features would be relevant to the two regression models
#which features have the most correlation with the two target variables (price and review rating)?

#In terms of top four five numerical features that correlate with price:
#accommodates 0.472709, bedrooms 0.465836, beds 0.397377, availability_30  0.121476, review_scores_location 0.116011, availability_60 0.09
#Note that categorical features not included

corr_matrix = df.corr()
corr_matrix
corr_matrix["price"].sort_values(ascending=False)

In [103]:
#Correlation of features for numercal variables
data_selected = df2[['price','review_scores_rating','number_of_reviews', 'accommodates', 'bedrooms', 'beds', 'availability_30','number_of_reviews','reviews_per_month','calculated_host_listings_count','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','host_total_listings_count','host_listings_count','minimum_nights', 'maximum_nights']]
colormap = plt.cm.RdB
plt.figure(figsize=(32,10))
plt.title('Correlation of Features', y=1.05, size=15)
sns.heatmap(data_selected.corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [104]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = data_selected.corr()
#cor = df2.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [105]:
#Look at the correlation between numerical variables to see what features would be relevant to the two regression models
#which features have the most correlation with the two target variables (price and review rating)?

#In terms of top four five numerical features that correlate with review rating score:
#accommodates 0.472709, bedrooms 0.465836, beds 0.397377, availability_30  0.121476, review_scores_location 0.116011, availability_60 0.09
#Note that categorical features not included

#Price not much related to review scores rating

corr_matrix = df.corr()
corr_matrix
corr_matrix["review_scores_rating"].sort_values(ascending=False)

In [106]:
#Plot of heatmaps of the correlations between different features
plt.subplots(figsize=(10,8))
corr = df.corr()
sns.heatmap(corr,  annot = True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm',linewidths=3, linecolor='black')
plt.show()

In [107]:
#Are prices or review ratings related to location?
df(kind="scatter", x="longitude", y="latitude", alpha=0.4,
s=df2["host_listings_count"]/100, label="Host listings", figsize=(10,7),
c="price", cmap=plt.get_cmap("jet"), colorbar=True,
)
plt.legend()

In [108]:
#Some explanatory analysis here, perhaps start with graphing listings by neighborhood
#plot the number of Airbnb listings in Toronto neighbourhoods
plt.figure(figsize=(10, 20), dpi=100 )
df['former_city'].value_counts().plot(kind = 'barh')
plt.xlabel("Count of Listings")
plt.ylabel("Neighbourhood Group")
plt.title("Number of Airbnb Listings in Toronto Neighbourhoods");


In [109]:
#Price by neighborhood
plt.figure(figsize=(20, 8), dpi=200 )
sns.barplot(x='former_city',y='price',data=df, palette='rainbow')
#x =  (df_listings_num['neighbourhood_cleansed'].value_counts()/df_listings_num.shape[0]).index
#y = round((df_listings_num['neighbourhood_cleansed'].value_counts()/df_listings_num.shape[0])*100, 1)
#sns.barplot(x= df2['neighbourhood_cleansed'], y=df2['price'], order = df2['price'].sort_values(ascending=False).index, palette="viridis", alpha=1)
plt.title("Price of listing by neighbourhood")

#sns.barplot(x='waterfront',y='price',data=Housingdata, palette='rainbow')
#plt.title("Price of house by presence of waterfront")

In [110]:
# calculate average price by listing in each neighbourhood
df.groupby('former_city').mean().sort_values('price', ascending = False)[['price']]
#Highest:Bridle Path-Sunnybrook-York Mills	850.000000
#Lowest:New Toronto	35.500000
#Why different from what is shown in Tableau?

In [111]:
#Calculate average review rating by listing in each neighborhood
df.groupby('former_city').mean().sort_values('review_scores_rating', ascending = False)[['review_scores_rating']]

In [112]:
#Find the room type distribution percentage
df_room = pd.DataFrame()
df_room['Room_Type_count'] = df.room_type.value_counts(ascending = False)
df_room['Room_Type_percentage'] = round(df.room_type.value_counts(ascending = False)/df2.shape[0]*100, 2)
df_room

In [113]:
#Look at the correlation between numerical variables to see what features would be relevant to the two regression models
#which features have the most correlation with the two target variables (price and review rating)?

#In terms of top four five numerical features that correlate with price:
#accommodates 0.472709, bedrooms 0.465836, beds 0.397377, availability_30  0.121476, review_scores_location 0.116011, availability_60 0.09
#Note that categorical features not included

corr_matrix = df.corr()
corr_matrix
corr_matrix["price"].sort_values(ascending=False)

In [114]:
df.info()

In [115]:
#Drop host_since, host_since_year, host_response_rate; these three variables are redundant and host_response rate has all 100 as response rate
df[['former_city','room_type']]

In [116]:
df=df.drop(['host_since','host_since_year','host_response_rate'],axis=1)

In [117]:
df.head(2)

# Linear Regression models with Target variable being Price and in another Linear Regression model Target variable being Review score ratings

In [118]:
#Import necessary libraries for performing prediction
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [119]:
#Dummy coding categorical variables
#https://medium.com/analytics-vidhya/implementing-linear-regression-using-sklearn-76264a3c073c
X = pd.get_dummies(df, drop_first=True)
X.head(5)


In [120]:
df1=X

In [121]:
#uint8(3318)
X.info()

In [122]:

#As all categorical variables were coded numerically as 0 and 1, transformation of categorical data to numeric is not necessary

#Model Building
#Investigate relationship of variables to price in linear regression with measure of strength of prediction being R2
#Start with linear regression
#Then multiple regression

#With only sqft of living space as predictive feature, R square value was 0.49
dfa = df[['accommodates']]
Y = df['price']
lm1 = LinearRegression()
lm1.fit(X, Y)

#print("The predicted values are : " + str(lm1.predict(X)))

print("The R^2 value for the linear regression model is : " + str(lm1.score(X, Y)))

In [123]:
#With additional features, the R2 value increased to 0.66
features = ['review_scores_rating', 'accommodates', 'bedrooms', 'beds', 'availability_30','number_of_reviews','reviews_per_month','calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms'] 

Y = df['price']
lm2 = LinearRegression()
lm2.fit(df[features], Y)

#print("The predicted values are mentioned as : " + str(lm2.predict(Housingdata[features])))

print("The R^2 value is : " + str(lm2.score(df[features], Y)))

In [124]:
X.info()

In [125]:
XfeaturedScaled = X

# Standard Linear Regression with selected variables from Correlation Matrix

In [126]:
import statsmodels.api as sm
model = sm.OLS.from_formula("price ~ accommodates + beds + bedrooms + host_length +host_acceptance_rate+host_is_superhost_t+review_scores_rating",data = X)
result = model.fit()
result.summary()

# MODELLING AND MODEL EVALUATION
# Split with 80%, 70%, 50% on training set and see effect on Model evaluation metrics RMSE and so forth
#Target being Price

In [127]:
#Fitting Machine Learning Model in this case being Linear regression
X = X.drop('price',axis=1)
#X = df2[features]
y = df['price']

#import necessary modules
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

#Create the training and the test set
#In this case, 85% of data allocated to training set
xtrain, xtest, ytrain, ytest = train_test_split(X,y,train_size=0.85,random_state=42)

xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

#Create the regressor
lm2 = LinearRegression()
lm2.fit(xtrain, ytrain)

predicted = lm2.predict(xtest)

#MODEL EVALUATION
from sklearn.metrics import adjusted_rand_score, r2_score, mean_squared_error, mean_absolute_error
print("R2 (explained variance):", round(r2_score(ytest, predicted), 2))
print("Mean Absolute Prediction Error (Σ(|y-pred|/y)/n):", round(np.mean(np.abs((ytest-predicted)/predicted)), 2))
print("Mean Absolute Error (Σ|y-pred|/n):", "{:,.0f}".format(mean_absolute_error(ytest, predicted)))
print("Root Mean Squared Error (sqrt(Σ(y-pred)^2/n)):", "{:,.0f}".format(np.sqrt(mean_squared_error(ytest, predicted))))

theta_1 = lm2.coef_[0]
theta_0 = lm2.intercept_
print(' y = {0} + x * {1}'.format(theta_0, theta_1))

In [128]:
#To compare the actual oputput values of cr_x_test with predicted values
ytest_predict = lm2.predict(xtest)
df = pd.DataFrame({'Actual':ytest, 'Predicted':ytest_predict})
df

In [ ]:
#Way to plot the predictions against actual to look at the difference?
#predictions = model.predict(xtest)
#sns.regplot(ytest,predictions)
plt.figure(figsize=(19, 8), dpi=300)
plt.subplot(2,1,1)
#plt.plot(y_test - lm_test_pred, marker='o',linestyle='', color =base)
plt.plot(ytest - ytest_predict, marker='o',linestyle='')
#plt.ylabel('Actual and Pred Price Difference-LM Model');
plt.title("Difference Between Actual and Predicted prices using LinearRegression model", fontsize = 15, weight = 'bold')
plt.yticks(fontsize = 12)
plt.xticks([])
plt.ylim(-500, 1700)

In [129]:
##Create the training and the test set
##In this case, 75% of data allocated to training set
xtrain, xtest, ytrain, ytest = train_test_split(X,y,train_size=0.50,random_state=42)

xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

#Create the regressor
lm2 = LinearRegression()
lm2.fit(xtrain, ytrain)

predicted = lm2.predict(xtest)

#MODEL EVALUATION
print("R2 (explained variance):", round(r2_score(ytest, predicted), 2))
print("Mean Absolute Perc Error (Σ(|y-pred|/y)/n):", round(np.mean(np.abs((ytest-predicted)/predicted)), 2))
print("Mean Absolute Error (Σ|y-pred|/n):", "{:,.0f}".format(mean_absolute_error(ytest, predicted)))
print("Root Mean Squared Error (sqrt(Σ(y-pred)^2/n)):", "{:,.0f}".format(np.sqrt(mean_squared_error(ytest, predicted))))
print(' y = {0} + x * {1}'.format(theta_0, theta_1))

In [130]:
#To compare the actual oputput values of cr_x_test with predicted values
#cr_y_test_predict = lm.predict(cr_x_test)
#df = pd.DataFrame({'Actual':cr_y_test, 'Predicted':cr_y_test_predict})
#df

ytest_predict = lm2.predict(xtest)
df = pd.DataFrame({'Actual':ytest, 'Predicted':ytest_predict})
df

In [131]:
sns.regplot(ytest,ytest_predict)

# STANDARD MULTIPLE REGRESSION

# Look at Coefficients, p value

In [132]:
#For standard multiple regression
#Note the coefficent how large it is
import statsmodels.api as sm
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

In [133]:
sns.regplot(y_test,predictions)

In [134]:
# plot feature importance by variable ??
feat_importances = pd.Series(model.feature_importances, index=x.columns).sort_values()
feat_importances.nlargest(10).plot(kind = 'barh')

# MODELLING AND MODEL EVALUATION
# Split with 80%, 70%, 50% on training set and see effect on Model evaluation metrics RMSE and so forth
# Target being Review Score Rating


In [135]:
df1

In [136]:
dfX=df1

In [137]:
#Fitting Machine Learning Model in this case being Linear regression
dfX = dfX.drop('review_scores_rating',axis=1)
#X = df2[features]
Y = df1['review_scores_rating']

#import necessary modules
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

#Create the training and the test set
#In this case, 85% of data allocated to training set
xtrain, xtest, ytrain, ytest = train_test_split(dfX,Y,train_size=0.85,random_state=42)

xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

#Create the regressor
lm2 = LinearRegression()
lm2.fit(xtrain, ytrain)

predicted = lm2.predict(xtest)

#MODEL EVALUATION
from sklearn.metrics import adjusted_rand_score, r2_score, mean_squared_error, mean_absolute_error
print("R2 (explained variance):", round(r2_score(ytest, predicted), 2))
print("Mean Absolute Prediction Error (Σ(|y-pred|/y)/n):", round(np.mean(np.abs((ytest-predicted)/predicted)), 2))
print("Mean Absolute Error (Σ|y-pred|/n):", "{:,.0f}".format(mean_absolute_error(ytest, predicted)))
print("Root Mean Squared Error (sqrt(Σ(y-pred)^2/n)):", "{:,.0f}".format(np.sqrt(mean_squared_error(ytest, predicted))))

theta_1 = lm2.coef_[0]
theta_0 = lm2.intercept_
print(' y = {0} + x * {1}'.format(theta_0, theta_1))

# Random Forest Regressor

In [138]:
#Create a RandomForestRegressor model to predict the prices
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)    
rf_model.fit(xtrain, ytrain)
rf_pred = rf_model.predict(xtest)

# Predict the property prices using the RandomForestRegressor model
rf_train_pred = rf_model.predict(xtrain)
rf_test_pred = rf_model.predict(xtest)

df = pd.DataFrame()
#df['LM_Predicted_Price'] = lm_test_pred
df['RF_Predicted_Price'] = rf_test_pred
df['Actual_y_test'] = y_test
df.tail(10)

# APPLY FEATURE SCALING THEN LINEAR REGRESSION; IS THERE A DIFFERENCE?

In [ ]:
#Is there some feature scaling such as with PCA required for the machine learning algorithm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Applying scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking','price']
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

df_train

# Clustering and Linear Regression

In [ ]:
#Clustering, then regression
#What features would put in clustering machine learning model?
#Perhaps those features most correlated with target variable or are the most predictive of target variable?



In [ ]:
#Code on PCA an  clustering from course work below

#Is scaling necessary?
#Do need to scale as some values are in decimal and other numbers are in whole number format
#ale the features in your data before applying PCA
from sklearn.preprocessing import StandardScaler
std_X = StandardScaler().fit_transform(ALSdata)

In [ ]:
#Using SKLearn perform PCA on the dataset to reduce the dimensionality of such a high
#dimensional dataset. (5 marks)
#How many components are enough to explain almost all of the data variance?
#Show the percentage of variance explained by each of the selected components

from sklearn.decomposition import PCA

#assuming components are 8
pca = PCA(n_components=8)
principalComponents = pca.fit_transform(std_X)
principalDf = pd.DataFrame(data = principalComponents,columns = ['principal component 1', 'principal component 2', 'principal component 3','principal component 4','principal component 5', 'principal component 6', 'principal component 7', 'principal component 8'])
principalDf

In [ ]:
#8 components does not explain all the variance
pca.explained_variance_ratio_

In [ ]:
#looks like 80 components explain the variance
pca = PCA().fit(std_X)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
pca = PCA(n_components=80)
principalComponents = pca.fit_transform(std_X)
pca.explained_variance_ratio_

In [ ]:
#https://www.integratedots.com/determine-number-of-iris-species-with-k-means/
Sum_of_squared_distances = []
K = range(1,10)
optimalK = 1
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(ALSdata)
    Sum_of_squared_distances.append(km.inertia_)
    if k >1:
       ratio = Sum_of_squared_distances[k-1]/Sum_of_squared_distances[k-2]
       if ratio < 0.55:
          optimialK = k

In [ ]:
#Plot Elbow graph
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method for Optimal k')
plt.show()

In [ ]:
#From elbow method chose 2 as number of clusters
from sklearn.cluster import KMeans
kmean = KMeans(n_clusters = 2)

Records_to_fit = ALSdata
Records_to_fit.sample(5)

In [ ]:
#https://blog.floydhub.com/introduction-to-k-means-clustering-in-python-with-scikit-learn/
#Specify the number of clusters (3) and fit the data X
#kmeans = KMeans(n_clusters=3, random_state=0).fit(X)

kmean.fit(Records_to_fit)

In [ ]:
centroids = kmean.cluster_centers_
labels=kmeans.labels_
print("Shape of Centroids Array: " + str(centroids.shape))

print(labels)
print(centroids)

In [ ]:
#Code below prints cluster number and percentage of data contains
#e.g., Cluster 0 contains 1147 samples with percentage of 49.85%
#Cluster 1 contains 1154 samples with percentage of 50.15%
from collections import Counter

labels = kmean.labels_
c = Counter(labels)
print(c.most_common())

for cluster_number in range(0,2):
  print("Cluster {} contains {} samples with percentage of {:.2f}%".format(
      cluster_number, c[cluster_number], c[cluster_number]/sum(c.values()) * 100))

In [ ]:
# Plotting the cluster centers and the data points on a 2D plane
plt.scatter(X[:, 0], X[:, -1])
    
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', marker='x')
    
plt.title('Data points and cluster centroids')
plt.show()

In [ ]:
#How to filter the clusters? Use aggregation?
#Aggregate the centroids according to labels?
#https://blog.floydhub.com/introduction-to-anomaly-detection-in-python/

#Apply multiple regression to each cluster


In [ ]:
#The Silhouette Coefficient is a measure of how well samples are clustered with samples that are similar to themselves. Clustering models with a high Silhouette Coefficient 
#are said to be dense, where samples in the same cluster are similar to each other, and well separated, where samples in different clusters are not very similar to each other.
from sklearn.metrics import silhouette_score as ss

In [ ]:
#As per SKlearn docs: The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a 
#sample has been assigned to the wrong cluster, as a different cluster is more similar.
mean_sihouette_score = ss(Record_array, labels)
print(mean_sihouette_score)

In [ ]:
#Try different cluster sizes
kmean = KMeans(n_clusters = 7)
Records2_to_fit = ALSdata
Records2_to_fit.sample(5)
kmean.fit(Records2_to_fit)

In [ ]:
from collections import Counter

labels = kmean.labels_
c = Counter(labels)
print(c.most_common())

for cluster_number in range(0,7):
  print("Cluster {} contains {} samples with percentage of {:.2f}%".format(
      cluster_number, c[cluster_number], c[cluster_number]/sum(c.values()) * 100))

In [ ]:
mean_sihouette_score = ss(Record2_array, labels)
print(mean_sihouette_score)

In [ ]:
#Create an empty array of mean silhouette scores
silhouette_scores = [] 

#Implement for loop, running kmeans on clusters ranging from 2 to 50 in the ALSdata
for n_cluster in range(2, 50):
    silhouette_scores.append(ss(ALSdata, KMeans(n_clusters = n_cluster).fit_predict(ALSdata)))
    print(silhouette_scores)

In [ ]:
#Line graph

#https://www.kaggle.com/vipulgandhi/kmeans-detailed-explanation
import matplotlib.pyplot as plt

#Create an empty array of mean silhouette scores
silhouette_scores = [] 

#Implement for loop, running kmeans on clusters ranging from 2 to 50 in the ALSdata
for n_cluster in range(2, 50):
    silhouette_scores.append(ss(ALSdata, KMeans(n_clusters = n_cluster).fit_predict(ALSdata))) 
    
# Plotting a line graph, plotting mean silhouette score with number of clusters
k = [2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49] 

plt.plot(k, silhouette_scores, color='green', linestyle='dashed', linewidth = 2, marker='o', markeredgecolor = 'green', markerfacecolor='red')
plt.title('K-Means')
plt.xlabel('Number of Clusters')
plt.ylabel('Mean Silhouette Score')
plt.show()

In [ ]:
silhouette_scores = [] 

for n_cluster in range(2, 50):
    silhouette_scores.append(ss(ALSdata, KMeans(n_clusters = n_cluster).fit_predict(ALSdata))) 
    
# Plotting a bar graph to compare the results 
k = [2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49] 
plt.bar(k, silhouette_scores) 
plt.xlabel('Number of clusters', fontsize = 10) 
plt.ylabel('Mean Silhouette Score', fontsize = 10) 
plt.show()

# Some more explanatory Analysis

In [ ]:
#Some explanatory analysis here, perhaps start with graphing listings by neighborhood
# plot the number of Airbnb listings in Toronto neighbourhoods
df2['neighbourhood_cleansed'].value_counts().plot(kind = 'barh')
plt.xlabel("Count of Listings")
plt.ylabel("Neighbourhood Group")
plt.title("Number of Airbnb Listings in Toronto Neighbourhoods");

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x='host_neighbourhood',y='price',data=df2, palette='rainbow')
plt.title("Price of listing by neighbourhood")

#sns.barplot(x='waterfront',y='price',data=Housingdata, palette='rainbow')
#plt.title("Price of house by presence of waterfront")

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


Now you're ready to read in the data and use the plotting functions to visualize the data.

Let's take a quick look at what the data looks like:

In [ ]:
df1.head(5)

Distribution graphs (histogram/bar graph) of sampled columns:

In [ ]:
plotPerColumnDistribution(df1, 10, 5)

Correlation matrix:

In [ ]:
plotCorrelationMatrix(df1, 9)

In [ ]:
plotCorrelationMatrix(df4_numerical_imputed, 9)

Scatter and density plots:

In [ ]:
plotScatterMatrix(df1, 20, 10)

In [ ]:
plotScatterMatrix(df4_numerical_imputed, 20, 10)